[An brief introduction on how to perform feature selection with scikit-feature](https://jundongl.github.io/scikit-feature/tutorial.html)

In [1]:
!python --version

Python 2.7.15


In [2]:
# import sys
# print ('\n'.join(sys.path))


/home/jovyan/work/tests
/opt/conda/envs/python2/lib/python27.zip
/opt/conda/envs/python2/lib/python2.7
/opt/conda/envs/python2/lib/python2.7/plat-linux2
/opt/conda/envs/python2/lib/python2.7/lib-tk
/opt/conda/envs/python2/lib/python2.7/lib-old
/opt/conda/envs/python2/lib/python2.7/lib-dynload
/opt/conda/envs/python2/lib/python2.7/site-packages
/opt/conda/envs/python2/lib/python2.7/site-packages/IPython/extensions
/home/jovyan/.ipython


In [5]:
# sys.path.append('/usr/local/lib/python2.7/dist-packages/')


In [7]:
# !pip install numpy

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
     |████████████████████████████████| 17.0 MB 3.7 MB/s eta 0:00:01


In [12]:
# !pip install scikit-learn==0.20.4

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
     |████████████████████████████████| 5.5 MB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 24.8 MB 3.8 MB/s eta 0:00:011


In [3]:
import scipy.io

In [4]:
mat = scipy.io.loadmat("COIL20.mat")

X=mat['X']
print(X)

y = mat['Y'][:, 0] 
print(y)

import numpy as np
n_labels = np.shape(y)
print(n_labels)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

[[0.01568627 0.01568627 0.01568627 ... 0.01568627 0.01568627 0.01568627]
 [0.01960784 0.01960784 0.01960784 ... 0.01960784 0.01960784 0.01960784]
 [0.01568627 0.01568627 0.01568627 ... 0.01568627 0.01568627 0.01568627]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[ 1  1  1 ... 20 20 20]
(1440,)


In [5]:
from skfeature.function.similarity_based import fisher_score

In [6]:
from skfeature.utility import unsupervised_evaluation

import numpy as np

In [7]:
score = fisher_score.fisher_score(X_train, y_train)
print(score)

[13.96904931  0.5376816   0.19923194 ...  3.71944606 14.01720752
 14.05075518]


In [8]:
idx = fisher_score.feature_ranking(score)

In [9]:
num_fea = 5
selected_features_train = X_train[:, idx[0:num_fea]]
selected_features_test = X_test[:, idx[0:num_fea]]
print(selected_features_train)

[[0.         0.         0.         0.         0.        ]
 [0.02352941 0.02352941 0.02352941 0.02352941 0.02352941]
 [0.         0.         0.         0.         0.        ]
 ...
 [0.01568627 0.01568627 0.01568627 0.01568627 0.01568627]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]]


In [10]:
from sklearn import svm

clf = svm.LinearSVC()


In [11]:
clf.fit(selected_features_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [12]:
y_predict = clf.predict(selected_features_test)
print(y_predict)

[8 8 2 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 2 8 8 8 8 8 8
 8 8 2 8 8 8 8 2 8 2 2 8 8 8 8 2 8 8 8 8 8 8 8 8 8 8 8 8 2 8 8 2 8 8 8 8 8
 8 8 2 8 8 8 8 8 8 8 8 8 8 8 8 2 2 8 8 8 8 8 2 8 2 8 8 8 8 2 2 5 8 8 8 8 8
 8 2 8 8 8 8 8 2 8 8 8 8 8 2 8 1 8 8 2 8 8 8 8 8 2 2 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 2 8 8 8 8 8 8 8 8 8 2 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 2 8 8 8 8 8 8 8 8 8 8 8 2 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 2 8 8 8 2 8 8
 8 8 2 8 8 8 8 2 8 8 8 8 8 8 8 2 8 8 8 8 8 8 8 8 8 8 8 8 8]


In [13]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_predict)
print(acc)

0.09375


In [14]:
from skfeature.utility import construct_W

kwargs_W = {"metric":"euclidean","neighbor_mode":"knn","weight_mode":"heat_kernel","k":5,'t':1}

W = construct_W.construct_W(X, **kwargs_W)

# Compute and output the score of each feature

from skfeature.function.similarity_based import lap_score

score = lap_score.lap_score(X, W=W)

print(score)

[0.01269462 0.00637613 0.00333286 ... 0.0123851  0.01271441 0.01269681]


In [15]:
idx = lap_score.feature_ranking(score)

In [16]:
from skfeature.utility import unsupervised_evaluation

import numpy as np

num_cluster = len(np.unique(y))

In [17]:
num_fea = 5
selected_features = X[:, idx[0:num_fea]]

In [18]:
print(num_cluster)

nmi,acc=unsupervised_evaluation.evaluation(X_selected=selected_features,n_clusters=num_cluster,y=y)

print(nmi)

print(acc)

20
0.41775691807264564
0.19305555555555556


/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


In [19]:
from skfeature.utility import unsupervised_evaluation
import numpy as np
num_cluster = len(np.unique(y))
print(num_cluster)
nmi,acc=unsupervised_evaluation.evaluation(X_selected=selected_features,n_clusters=num_cluster,y=y)
print (nmi)
print (acc)

20
0.4178036862998123
0.1951388888888889
